# JSON en Python

In [5]:
import json


un_entero = json.loads('1')

una_cadena = json.loads('"this is a string"')

un_array = json.loads('[1, 2, 3]')

un_objeto = json.loads('{"key": "value", "pages": [1, 2, 3, 65], "age": 22, "height": 1.85}')

print(un_entero)
print(una_cadena)
print(un_array)
print(un_objeto)

1
this is a string
[1, 2, 3]
{'key': 'value', 'pages': [1, 2, 3, 65], 'age': 22, 'height': 1.85}


## Repaso de Python


### Acceder a un valor simple
Si el parsing se hizo correctamente, la variable contendrá el valor

In [2]:
print("el entero:", un_entero)
print("la cadena:", una_cadena)


el entero: 1
la cadena: this is a string


### Acceder a un array
Un elemento de array se puede acceder a tráves de su posición

In [9]:
print('El array entero:', un_array)
print('El primer elemento:', un_array[0]) # Indexes in python start from 0
print('El último elemento:', un_array[2]) # There's 3 items, so indexes are 0, 1, 2
print('El último elemento, en otra manera:', un_array[-1])
un_array[3]

El array entero: [1, 2, 3]
El primer elemento: 1
El último elemento: 3
El último elemento, en otra manera: 3


IndexError: list index out of range

### Acceder a un objeto
Un objeto de JSON en python se parsea como un diccionario. Si el objeto contiene otros objetos, esos también serán sub-diccionarios 

In [10]:
print('El objeto entero:', un_objeto)
print('El valor con nombre "key":', un_objeto['key'])
print('El último elemento del array "pages":', un_objeto['pages'][-1])
print('El valor "height":', un_objeto['height'])

El objeto entero: {'key': 'value', 'pages': [1, 2, 3, 65], 'age': 22, 'height': 1.85}
El valor con nombre "key": value
El último elemento del array "pages": 65
El valor "height": 1.85


## Gestionar errores

El input puede contener un JSON invalido; esta situación producirá un error en Python, y sin una gestión del error, el programa fallará inmediatamente. 

Para evitar esta situación, podemos incrustar el comando `json.loads` en un bloque de tipo `try`/`except`

In [11]:
from json.decoder import JSONDecodeError
a_tentative_json = "123abc" # this is not valid as number (spurious characters), nor as string (no quotes)

try:
  parsed = json.loads(a_tentative_json)
  print("input procesado con éxito: ", parsed)
except JSONDecodeError: # we're being specific, this will catch only this exception, any other will make the program to flow
  print(f"algo no ha ido bien procesando el input: '{a_tentative_json}'")

algo no ha ido bien procesando el input: '123abc'


### Handling all errors

Las excepciones en Python forman una hierarquía que tiene `Exception` como raíz. Esto significa que utilizando `Exception` en la cláusula de `except` podemos capturar practicamente todos los errores. Al mismo tiempo, esto significa que todos los errores se procesarán como si fueran el mismo: esto no nos importa mucho ahora, pero en general no es una buena práctica. 


Ejemplo:
```
try:
  # do something
except Exception as e:
  print(f"Something happened! {e}")
```  

# Data classes

## Modelar un JSON en una data class de Python

### Crear una data class

Una dataclass de python contiene datos inmutables. Por ejemplo, la siguiente clase define una Persona con dos atributos: el nombre y el apellido. 

In [44]:
from dataclasses import dataclass

@dataclass
class Person:
  """Class to model a Person"""
  first_name: str
  last_name: str

### Parsear el input

En este caso, simulamos un fichero con datos json a través de un array que contiene dos objetos de JSON. 

Observemos que el JSON puede contener más atributos de los que hemos modelado. 

In [45]:
import json

lines = ['{"first_name": "Luca", "last_name": "Telloli", "role": "professor", "age": 25}',
         '{"first_name": "Ana", "last_name": "Freire", "role": "coordinator", "age": 25}']

data = []

def parse_line(line: str):
  """Try to parse a string into a Person"""
  try:
    parsed = json.loads(line)
    return Person(parsed['first_name'], parsed['last_name']) # Use data class, we only extract from JSON the relevant fields
  except Exception as e:
    print(f"Something happened with line '{line}': {e}")

def parse_line2(line: str):
  """Try to parse a string into a Person"""
  try:
    parsed = json.loads(line)
    if(parsed['first_name'] and parsed['last_name']):
        return Person(parsed['first_name'], parsed['last_name']) # Use data class, we only extract from JSON the relevant fields
    else:
        return None
  except Exception as e:
    print(f"Something happened with line '{line}': {e}")

for line in lines:
  data_item = parse_line(line)
  data.append(data_item)

print(data)

[Person(first_name='Luca', last_name='Telloli'), Person(first_name='Ana', last_name='Freire')]


### Casos límite (Corner cases)

El input puede contener valores incorrectos (dirty values) o ser incompleto. 

Que hacer en estos casos? Unas opciones posibles son: 
* ignorar estos valores
* intentar "salvar" la mayoría de información 
* fallar la aplicación cuando encontramos un valor así 

In [50]:
lines = ['{"first_name": "Luca", "last_name": null}',
         '{"first_name": "Ana"}',
         '{"first_name": "Mario", "last_name": "Rossi"}'
        ]

data = []

for line in lines:
  parsed = parse_line2(line)
  if parsed:  
      data.append(parsed)

print("*** PARSED DATA ***", "\n")
print(data)

Something happened with line '{"first_name": "Ana"}': 'last_name'
*** PARSED DATA *** 

[Person(first_name='Mario', last_name='Rossi')]


# ~Question 1.1~

Que cambios tendríamos que implementar en nuestro código si no queremos que apareza el valor `None` en nuestro array `data`? Donde y de que manera hay que cambiar el código? 

Añade una celda aquí abajo con tu solución

**NOTA** Este ejercicio lo hicimos en clase, así que no hace falta contestar esta pregunta.

# Question 1

## I Parte 
Dado como input el siguiente:

```
lines = ['{"first_name": "Luca", "last_name": "Telloli", "role": "professor", "age": 25}',
         '{"first_name": "Ana", "last_name": "Freire", "role": "coordinator", "age": 25}']
```
Vamos a considerar otro atributo en nuestro input, `age`: es decir, queremos que la edad también apareza como información dentro de nuestro array `data`

1. Como cambia la data class?
2. Cómo cambia el código de parsing?

Añade una celda abajo donde reimplementas el método `parse_line(line: str)` y lo usas de una forma similar a la de los ejemplos de arriba. 

## II Parte 

Amplía el input añadiendo un par más de valores de ejemplo. 

In [31]:
# Respuesta Parte 1
from dataclasses import dataclass

@dataclass #Python decorator
class Person: # Define Person class
  """Class to model a Person"""
  first_name: str
  last_name: str
  age: int # number in JSON  

In [32]:
import json    
from json.decoder import JSONDecodeError

# LIST
lines = ['{"first_name": "Luca", "last_name": "Telloli", "role": "professor", "age": 25}',
         '{"first_name": "Ana", "last_name": "Freire", "role": "coordinator", "age": 25}']

data = []

# FOR LOOP
for line in lines:
    try:
        # PARSE IT
        person_lines = json.loads(line) # JSON.LOADS -> PARSES THE STRING & CONVERTS IT INTO A PYTHON DICTIONARY 
        # OBJECT -> person
        person = Person(
            first_name=person_lines['first_name'],# DICT
            last_name=person_lines['last_name'],# DICT
            age=person_lines['age']# DICT
        )
        data.append(person)
    #WILL GET THIS ERROR WHEN A DATA TYPE IS INCOMPATIBLE   
    except JSONDecodeError:
        # F'STRINGS 
        print(f"Error parsing line: '{line}'")

print(data)

[Person(first_name='Luca', last_name='Telloli', age=25), Person(first_name='Ana', last_name='Freire', age=25)]


or using a parsing function:

In [39]:
import json    
from json.decoder import JSONDecodeError

# LIST
lines = ['{"first_name": "Luca", "last_name": "Telloli", "role": "professor", "age": 25}',
         '{"first_name": "Ana", "last_name": "Freire", "role": "coordinator", "age": 25}']

data = []
# USING parse_line(line: str)
def parse_line(line: str):# PARSING FUNCTION
    try:
        person_lines = json.loads(line)
        # OBJECT -> person
        person = Person(
            first_name=person_lines['first_name'],
            last_name=person_lines['last_name'],
            age=person_lines['age']
        )
        return person
    #WILL GET THIS ERROR WHEN A DATA TYPE IS INCOMPATIBLE   
    except JSONDecodeError:
        print(f"Error parsing line: '{line}'")
        return None # IF THE PARSING FAILS

for line in lines:
    person = parse_line(line)
    if person:
        data.append(person)

print(data)

[Person(first_name='Luca', last_name='Telloli', age=25), Person(first_name='Ana', last_name='Freire', age=25)]


Now doing it like the 2 examples above:

In [40]:
import json

# LIST
lines = ['{"first_name": "Luca", "last_name": "Telloli", "role": "professor", "age": 25}',
         '{"first_name": "Ana", "last_name": "Freire", "role": "coordinator", "age": 25}']

data = []

# BETTER ERROR HANDLING
def parse_line(line: str):  # PARSING FUNCTION 1
    """Try to parse a string into a Person"""
    try:
        parsed = json.loads(line)
        return Person(parsed['first_name'], parsed['last_name'], int(parsed['age'])) #TYPE CASTING
    except Exception as e:
        print(f"Something happened with line '{line}': {e}")
        return None

def parse_line2(line: str):  # PARSING FUNCTION 2
    """Try to parse a string into a Person"""
    try:
        parsed = json.loads(line)
        if parsed['first_name'] and parsed['last_name']:
            return Person(parsed['first_name'], parsed['last_name'], int(parsed['age'])) #TYPE CASTING
        else:
            return None # IF THE PARSING FAILS
    except Exception as e:
        print(f"Something happened with line '{line}': {e}")
        return None # IF THE PARSING FAILS

def parse_line3(line: str):  # PARSING FUNCTION 3
    """Try to parse a string/int into a Person"""
    try:
        parsed = json.loads(line)
        return Person(parsed['first_name'], parsed['last_name'], parsed['age'])
    except Exception as e:
        print(f"Something happened with line '{line}': {e}")
        return None # IF THE PARSING FAILS
    
for line in lines:
    data_item = parse_line3(line) #CORNER CASE
    if data_item:
        data.append(data_item)

print(data)

[Person(first_name='Luca', last_name='Telloli', age=25), Person(first_name='Ana', last_name='Freire', age=25)]


In [ ]:
Should we `typecast` or no?

In [54]:
# Respuesta Parte 2
from dataclasses import dataclass

@dataclass #Python decorator
class Person: # Define Person class
  """Class to model a Person"""
  first_name: str
  last_name: str
  age: int # number in JSON  
  is_verified: bool   

In [55]:
import json

lines = ['{"first_name": "Luca", "last_name": "Telloli", "role": "professor", "age": 25, "is_verified": false}',
         '{"first_name": "Ana", "last_name": "Freire", "role": "coordinator", "age": 25, "is_verified": true}']
data = []

# BETTER ERROR HANDLING
def parse_line(line: str):  # PARSING FUNCTION 1
    """Try to parse a string into a Person"""
    try:
        parsed = json.loads(line)
        return Person(parsed['first_name'], parsed['last_name'], parsed['age'], parsed['is_verified']) 
    except Exception as e:
        print(f"Something happened with line '{line}': {e}")
        return None

def parse_line2(line: str):  # PARSING FUNCTION 2
    """Try to parse a string into a Person"""
    try:
        parsed = json.loads(line)
        if parsed['first_name'] and parsed['last_name']:
            return Person(parsed['first_name'], parsed['last_name'], parsed['age'], parsed['is_verified'])
        else:
            return None # IF THE PARSING FAILS
    except Exception as e:
        print(f"Something happened with line '{line}': {e}")
        return None # IF THE PARSING FAILS

def parse_line3(line: str):  # PARSING FUNCTION 3
    """Try to parse a string into a Person"""
    try:
        parsed = json.loads(line)
        return Person(parsed['first_name'], parsed['last_name'], parsed['age'], parsed['is_verified'])
    except Exception as e:
        print(f"Something happened with line '{line}': {e}")
        return None # IF THE PARSING FAILS
    
def parse_line4(line: str):  # PARSING FUNCTION 4
    """Try to parse a string into a Person"""
    try:
        parsed = json.loads(line)
        return Person(parsed['first_name'], parsed['last_name'], parsed['age'], parsed['is_verified'])
    except Exception as e:
        print(f"Something happened with line '{line}': {e}")
        return None # IF THE PARSING FAILS
    
for line in lines:
    data_item = parse_line4(line) #CORNER CASE
    if data_item:
        data.append(data_item)

print(data)

[Person(first_name='Luca', last_name='Telloli', age=25, is_verified=False), Person(first_name='Ana', last_name='Freire', age=25, is_verified=True)]
